!pip install xmltodict
pip install requests
!pip install bs4
!pip install pandas

리눅스 gpu 확인

lspci | grep -i VGA


In [15]:
import requests
import xmltodict
from datetime import datetime, timedelta
import xmltodict
import os

import json
import sys
from bs4 import BeautifulSoup
import pandas as pd

## 폴더 내 모든파일 불러오기 예시

In [2]:
# 일별농업기상 pandas dataframe 생성

import os

path = './data/일별농업기상/'
file_list = os.listdir(path)
len(file_list)

962

In [8]:
with open(path+file_list[66], "r", encoding='utf-8') as json_file:
    json_data = json.load(json_file)
    data = json.dumps(json_data, indent = '\t', ensure_ascii=False)
d = json.loads(data)   
if d['response']['body']['items'] == None:
    print('holy')

In [9]:
df_base = pd.DataFrame(d['response']['body']['items']['item'])
df = pd.DataFrame(columns = df_base.columns)
df

,no,stn_Code,stn_Name,date,temp,max_Temp,min_Temp,hum,widdir,wind,rain,sun_Time,sun_Qy,condens_Time,gr_Temp,soil_Temp,soil_Wt


In [14]:
df = pd.DataFrame(columns = df_base.columns)

for i in range(0, len(file_list)):
    with open(path+file_list[i], "r", encoding='utf-8') as json_file:
        json_data = json.load(json_file)
        data = json.dumps(json_data, indent = '\t', ensure_ascii=False)
    d = json.loads(data)
    if d['response']['body']['items'] == None:
        continue
    else:
        df = df.append(d['response']['body']['items']['item'], ignore_index=True)
df = df.sort_values(by=['date'], axis=0)
df

,no,stn_Code,stn_Name,date,temp,max_Temp,min_Temp,hum,widdir,wind,rain,sun_Time,sun_Qy,condens_Time,gr_Temp,soil_Temp,soil_Wt
2871,1,412040A002,고양시 덕양구,2015-07-01,22.9,27.4,19.6,73.1,258.9,1.3,0,648,40,455,24.1,23.7,None
52930,1,618803A001,부산시 강서구,2015-07-01,22.3,26.5,19,93.8,200.7,1.9,0,None,34.4,582,None,21.9,25.7
16581,1,467800A001,이천시 중리동,2015-07-01,24.4,29.7,21,68.3,190,0.9,0,539,15.5,None,None,None,18.3
27660,1,339814A001,세종시 연서면,2015-07-01,22.4,27.2,18.2,75.4,237.2,2,0,None,17.1,0,24.3,-11.3,None
54131,1,621100A001,김해시 전하동,2015-07-01,22.7,27.4,18.8,80.6,267.8,1.6,0,490,25.3,100,None,21.7,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37642,1,039102B003,구미시 옥성면,2021-09-16,20.2,26.8,14.1,84.5,139.4,0.4,0.5,193,12.4,542,None,23,50
4564,1,055365F016,완주군 반교리,2021-09-16,23.1,29.8,16,65.8,80.9,0.3,1,None,17.8,272,None,None,None
27059,1,515804A001,장성군 장성읍,2021-09-16,21.3,27.6,15.3,81.7,59.8,0.9,0.5,251,10,405,22.4,24.3,27.5
50524,1,225874A001,횡성군 공근면,2021-09-16,19.5,28.4,11.8,77.4,84.8,0.7,0,513,19.8,0,20.2,22,22.7


In [16]:
df.to_csv('./일별농업기상_20150701_20210916.csv', encoding='cp949')

In [38]:
# ~16일 + 9월17~30일 data 추가생성
df_past = pd.read_csv("./일별농업기상_20150701_20210916.csv", encoding="cp949")
df_past = df_past.drop('Unnamed: 0', axis=1)

path = './data/일별농업기상/'
file_list = os.listdir(path)

n_file = []
dates = range(17,31)

for file in file_list:
    file = file.rstrip()
    for date in dates:
        if file.endswith('2021-09-{}.json'.format(date)):
            n_file.append(file)
n_file

In [24]:
with open(path+n_file[0], "r", encoding='utf-8') as json_file:
    json_data = json.load(json_file)
    data = json.dumps(json_data, indent = '\t', ensure_ascii=False)
d = json.loads(data)   
if d['response']['body']['items'] == None:
    print('holy')

df_base = pd.DataFrame(d['response']['body']['items']['item'], index=[0])
df = pd.DataFrame(columns = df_base.columns)

df = pd.DataFrame(columns = df_base.columns)

for i in range(0, len(n_file)):
    with open(path+n_file[i], "r", encoding='utf-8') as json_file:
        json_data = json.load(json_file)
        data = json.dumps(json_data, indent = '\t', ensure_ascii=False)
    d = json.loads(data)
    if d['response']['body']['items'] == None:
        continue
    else:
        df = df.append(d['response']['body']['items']['item'], ignore_index=True)
df = df.sort_values(by=['date'], axis=0)

df_recent = pd.concat([df_past, df])
df_recent.to_csv('./data/일별농업기상_20150701_20210930.csv', encoding='cp949')

In [209]:
for loca in df["stn_Name"].unique():
    print(loca,df.loc[df["stn_Name"] == loca, "sun_Qy"].isna().sum())

구미시 옥성면 108
김포시 월곶면 141
남양주 진건읍 6
부산시 강서구 3
신안군 압해읍 0
아산시 염치읍 0
양주시 광적면 6
영광군 군서면 4
완주군 반교리 880
용인시 처인구 68
진도군 군내면 2
청주시 남일면 1
평창군 여만리 3


In [202]:
# 데이터확인
file_path = "./data/일별농업기상/weather완주군_2016-01-01_2016-12-31_1.json"
with open(file_path, "r", encoding='utf-8') as json_file:
    json_data = json.load(json_file)
    data = json.dumps(json_data, indent = '\t', ensure_ascii=False)
    print(data)

{
	"response": {
		"header": {
			"result_Code": "200",
			"result_Msg": "OK"
		},
		"body": {
			"rcdcnt": "100",
			"page_No": "1",
			"total_Count": "365",
			"items": {
				"item": [
					{
						"no": "1",
						"stn_Code": "055365F016",
						"stn_Name": "완주군 반교리",
						"date": "2016-01-01",
						"temp": null,
						"max_Temp": null,
						"min_Temp": null,
						"hum": null,
						"widdir": null,
						"wind": null,
						"rain": null,
						"sun_Time": null,
						"sun_Qy": null,
						"condens_Time": null,
						"gr_Temp": null,
						"soil_Temp": null,
						"soil_Wt": null
					},
					{
						"no": "2",
						"stn_Code": "055365F016",
						"stn_Name": "완주군 반교리",
						"date": "2016-01-02",
						"temp": null,
						"max_Temp": null,
						"min_Temp": null,
						"hum": null,
						"widdir": null,
						"wind": null,
						"rain": null,
						"sun_Time": null,
						"sun_Qy": null,
						"condens_Time": null,
						"gr_Temp": null,
						"soil_Temp": null,
						"soil_Wt"

In [24]:
# 시장도매인 가격 pandas dataframe 생성
path = './data/시장도매인가격/'
file_list = os.listdir(path)
file_list.sort()

with open(path+file_list[1000], "r", encoding='utf-8') as json_file:
    json_data = json.load(json_file)
    data = json.dumps(json_data, indent = '\t', ensure_ascii=False)
d = json.loads(data)   
if d['lists']['list_total_count'] == 0:
    print('holy')

df_base = pd.DataFrame(d['lists']['list'])
columns = ['DELNG_DE'] + list(df_base.columns)
#columns = list(df_base.columns)
df = pd.DataFrame(columns = columns)
df

,DELNG_DE,PUM_NM,G_NAME,U_NAME,MI_P,MA_P,AV_P,PAV_P,FLUC,D_MARK,J_7,J_7_MARK


In [25]:
for i in range(0, len(file_list)):
    with open(path+file_list[i], "r", encoding='utf-8') as json_file:
        json_data = json.load(json_file)
        data = json.dumps(json_data, indent = '\t', ensure_ascii=False)
    d = json.loads(data)
    if d['lists']['list_total_count'] != '4':
        #print(file_list[i][4:12])
        #print(i, d['lists']['list_total_count'])
        continue
    elif d['lists']['list_total_count'] == '4':
        df = df.append(d['lists']['list'], ignore_index=True)
        df['DELNG_DE'].loc[len(df['DELNG_DE'])-4:len(df['DELNG_DE'])] = file_list[i][4:12]
df

,DELNG_DE,PUM_NM,G_NAME,U_NAME,MI_P,MA_P,AV_P,PAV_P,FLUC,D_MARK,J_7,J_7_MARK
0,20160919,대파(일반),특,1 kg단,"2,800","2,800","2,800",0,"2,800",.0,"2,281",122.8
1,20160919,대파(일반),상,1 kg단,"1,800","2,800","2,307",0,"2,307",.0,"1,982",116.4
2,20160919,대파(일반),보통,1 kg단,"1,170","1,800","1,582",0,"1,582",.0,"1,417",111.7
3,20160919,대파(일반),하,1 kg단,700,"1,170",917,0,917,.0,"1,041",88.1
4,20160920,대파(일반),특,1 kg단,"2,800","3,000","2,920","2,800",120,104.0,"2,781",105.0
...,...,...,...,...,...,...,...,...,...,...,...,...
6187,20210915,대파(일반),하,1 kg단,500,"1,400","1,132","1,013",119,112.0,"1,010",112.0
6188,20210916,대파(일반),특,1 kg단,"1,800","2,300","1,990","1,910",80,104.0,"1,950",102.1
6189,20210916,대파(일반),상,1 kg단,"1,500","1,800","1,605","1,570",35,102.0,"1,579",101.7
6190,20210916,대파(일반),보통,1 kg단,"1,440","1,500","1,449","1,445",4,100.0,"1,395",103.9


In [26]:
df.to_csv('./시장도매인가격20160919_20210916.csv',encoding='cp949')

In [50]:
# ~16일 + 9월17~30일 data 추가생성
df_past = pd.read_csv("./시장도매인가격20160919_20210916.csv", encoding="cp949")
df_past = df_past.drop('Unnamed: 0', axis=1)

path = './data/시장도매인가격/'
file_list = os.listdir(path)

n_file = []
dates = range(17,31)

for file in file_list:
    file = file.rstrip()
    for date in dates:
        if file.endswith('202109{}.json'.format(date)):
            n_file.append(file)
n_file

['시장도매20210917.json',
 '시장도매20210918.json',
 '시장도매20210919.json',
 '시장도매20210920.json',
 '시장도매20210921.json',
 '시장도매20210922.json',
 '시장도매20210923.json',
 '시장도매20210924.json',
 '시장도매20210925.json',
 '시장도매20210926.json',
 '시장도매20210927.json',
 '시장도매20210928.json',
 '시장도매20210929.json',
 '시장도매20210930.json']

In [57]:
with open(path+n_file[0], "r", encoding='utf-8') as json_file:
    json_data = json.load(json_file)
    data = json.dumps(json_data, indent = '\t', ensure_ascii=False)
d = json.loads(data)   
if d['lists']['list_total_count'] == 0:
    print('holy')

df_base = pd.DataFrame(d['lists']['list'])
columns = ['DELNG_DE'] + list(df_base.columns)
#columns = list(df_base.columns)
df = pd.DataFrame(columns = columns)
df

,DELNG_DE,PUM_NM,G_NAME,U_NAME,MI_P,MA_P,AV_P,PAV_P,FLUC,D_MARK,J_7,J_7_MARK


In [58]:
for i in range(0, len(n_file)):
    with open(path+n_file[i], "r", encoding='utf-8') as json_file:
        json_data = json.load(json_file)
        data = json.dumps(json_data, indent = '\t', ensure_ascii=False)
    d = json.loads(data)
    if d['lists']['list_total_count'] != '4':
        #print(file_list[i][4:12])
        #print(i, d['lists']['list_total_count'])
        continue
    elif d['lists']['list_total_count'] == '4':
        df = df.append(d['lists']['list'], ignore_index=True)
        df['DELNG_DE'].loc[len(df['DELNG_DE'])-4:len(df['DELNG_DE'])] = n_file[i][4:12]
df

,DELNG_DE,PUM_NM,G_NAME,U_NAME,MI_P,MA_P,AV_P,PAV_P,FLUC,D_MARK,J_7,J_7_MARK
0,20210917,대파(일반),특,1 kg단,"2,000","2,400","2,210","1,990",220,111.0,"1,959",112.8
1,20210917,대파(일반),상,1 kg단,"1,650","2,000","1,783","1,605",178,111.0,"1,572",113.4
2,20210917,대파(일반),보통,1 kg단,"1,200","1,650","1,302","1,449",-147,90.0,"1,404",92.7
3,20210917,대파(일반),하,1 kg단,800,"1,200","1,036","1,214",-178,85.0,"1,081",95.9
4,20210918,대파(일반),특,1 kg단,"2,200","2,400","2,281","2,210",71,103.0,"2,007",113.6
5,20210918,대파(일반),상,1 kg단,"1,730","2,200","1,859","1,783",76,104.0,"1,611",115.4
6,20210918,대파(일반),보통,1 kg단,"1,200","1,730","1,471","1,302",169,113.0,"1,402",104.9
7,20210918,대파(일반),하,1 kg단,800,"1,200","1,069","1,036",33,103.0,"1,087",98.3
8,20210920,대파(일반),특,1 kg단,"2,200","2,400","2,281",0,"2,281",.0,"2,059",110.8
9,20210920,대파(일반),상,1 kg단,"1,730","2,200","1,859",0,"1,859",.0,"1,665",111.6


In [59]:
df_recent = pd.concat([df_past, df])
df_recent


,DELNG_DE,PUM_NM,G_NAME,U_NAME,MI_P,MA_P,AV_P,PAV_P,FLUC,D_MARK,J_7,J_7_MARK
0,20160919,대파(일반),특,1 kg단,"2,800","2,800","2,800",0,"2,800",0.0,"2,281",122.8
1,20160919,대파(일반),상,1 kg단,"1,800","2,800","2,307",0,"2,307",0.0,"1,982",116.4
2,20160919,대파(일반),보통,1 kg단,"1,170","1,800","1,582",0,"1,582",0.0,"1,417",111.7
3,20160919,대파(일반),하,1 kg단,700,"1,170",917,0,917,0.0,"1,041",88.1
4,20160920,대파(일반),특,1 kg단,"2,800","3,000","2,920","2,800",120,104.0,"2,781",105.0
...,...,...,...,...,...,...,...,...,...,...,...,...
35,20210929,대파(일반),하,1 kg단,700,"1,000",972,"1,066",-94,91.0,"1,047",92.9
36,20210930,대파(일반),특,1 kg단,"1,730","3,500","2,230","1,916",314,116.0,"2,564",87.0
37,20210930,대파(일반),상,1 kg단,"1,600","1,730","1,727","1,730",-3,100.0,"1,993",86.7
38,20210930,대파(일반),보통,1 kg단,"1,000","1,600","1,153","1,384",-231,83.0,"1,503",76.7


In [62]:
df_recent.to_csv('./data/시장도매인가격_20160919_20210930.csv', encoding='cp949')

In [21]:
# 데이터확인
file_path = "./data/시장도매인가격/시장도매20210913.json"
with open(file_path, "r", encoding='utf-8') as json_file:
    json_data = json.load(json_file)
    data = json.dumps(json_data, indent = '\t', ensure_ascii=False)
    print(data)

{
	"lists": {
		"list_total_count": "4",
		"list": [
			{
				"PUM_NM": "대파(일반)",
				"G_NAME": "특",
				"U_NAME": "1 kg단",
				"MI_P": "1,900",
				"MA_P": "2,200",
				"AV_P": "2,003",
				"PAV_P": "0",
				"FLUC": "2,003",
				"D_MARK": ".0",
				"J_7": "1,890",
				"J_7_MARK": "106.0"
			},
			{
				"PUM_NM": "대파(일반)",
				"G_NAME": "상",
				"U_NAME": "1 kg단",
				"MI_P": "1,500",
				"MA_P": "1,900",
				"AV_P": "1,580",
				"PAV_P": "0",
				"FLUC": "1,580",
				"D_MARK": ".0",
				"J_7": "1,584",
				"J_7_MARK": "99.7"
			},
			{
				"PUM_NM": "대파(일반)",
				"G_NAME": "보통",
				"U_NAME": "1 kg단",
				"MI_P": "1,300",
				"MA_P": "1,500",
				"AV_P": "1,422",
				"PAV_P": "0",
				"FLUC": "1,422",
				"D_MARK": ".0",
				"J_7": "1,331",
				"J_7_MARK": "106.9"
			},
			{
				"PUM_NM": "대파(일반)",
				"G_NAME": "하",
				"U_NAME": "1 kg단",
				"MI_P": "900",
				"MA_P": "1,300",
				"AV_P": "1,106",
				"PAV_P": "0",
				"FLUC": "1,106",
				"D_MARK": ".0",
				"J_7": "970",
				"J_7_

In [29]:
# 도매시장경매가격 pandas dataframe 생성
path = './data/도매시장경매가격/'
file_list = os.listdir(path)

with open(path+file_list[62], "r", encoding='utf-8') as json_file:
    json_data = json.load(json_file)
    data = json.dumps(json_data, indent = '\t', ensure_ascii=False)
d = json.loads(data)   
if d['Grid_20180118000000000581_1']['row'] == None:
    print('holy')

df_base = pd.DataFrame(d['Grid_20180118000000000581_1']['row'])
df = pd.DataFrame(columns = df_base.columns)
df

,ROW_NUM,DELNG_DE,PBLMNG_WHSAL_MRKT_NM,PBLMNG_WHSAL_MRKT_CD,CPR_NM,CPR_CD,RISENO,BIDTIME,AUC_SE_CODE,AUC_SE_NM,...,STNDRD_CD,GRAD,GRAD_CD,SHIPMNT_SE_CODE,SHIPMNT_SE_NM,SANJI_CD,SANJI_NM,CPR_USE_SANJI_CD,CPR_USE_SANJI_NM,DELNG_QY


In [30]:
df = pd.DataFrame(columns = df_base.columns)

for i in range(0, len(file_list)):
    with open(path+file_list[i], "r", encoding='utf-8') as json_file:
        json_data = json.load(json_file)
        data = json.dumps(json_data, indent = '\t', ensure_ascii=False)
    d = json.loads(data)
    if d['Grid_20180118000000000581_1']['row'] == None:
        continue
    else:
        df = df.append(d['Grid_20180118000000000581_1']['row'], ignore_index=True)

df

,ROW_NUM,DELNG_DE,PBLMNG_WHSAL_MRKT_NM,PBLMNG_WHSAL_MRKT_CD,CPR_NM,CPR_CD,RISENO,BIDTIME,AUC_SE_CODE,AUC_SE_NM,...,STNDRD_CD,GRAD,GRAD_CD,SHIPMNT_SE_CODE,SHIPMNT_SE_NM,SANJI_CD,SANJI_NM,CPR_USE_SANJI_CD,CPR_USE_SANJI_NM,DELNG_QY
0,1001,20160423,정읍도매시장,350402,정읍원협(공),35040201,40,,2,정가수의,...,121100,없음,10,3,개별,580090,전북 정읍시 농소동,580090,전북 정읍시,3
1,1002,20160423,정읍도매시장,350402,정읍원협(공),35040201,20,,2,정가수의,...,121100,없음,10,3,개별,906000,광주광역시,906000,광주,10
2,1003,20160423,목포농산시장,360101,목포농수산,36010101,00001,,1,경매,...,121000,없음,10,3,개별,950-00,,,,100
3,1004,20160423,목포농산시장,360101,목포농수산,36010101,00001,,1,경매,...,121000,없음,10,3,개별,950-00,,,,110
4,1005,20160423,목포농산시장,360101,목포농수산,36010101,00001,,1,경매,...,121000,없음,10,3,개별,950-00,,,,135
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2590180,1577,20150910,진주도매시장,380401,진주중앙청과,38040102,0000000749,20150910080000,1,경매,...,121100,특,11,2,계통,740010,경북 김천시 모암동,3703,경북김천,50
2590181,1578,20150910,진주도매시장,380401,진주중앙청과,38040102,0000000750,20150910080000,1,경매,...,121100,특,11,2,계통,740010,경북 김천시 모암동,3703,경북김천,50
2590182,1579,20150910,진주도매시장,380401,진주중앙청과,38040102,0000000751,20150910080000,1,경매,...,121100,특,11,2,계통,740010,경북 김천시 모암동,3703,경북김천,50
2590183,1580,20150910,진주도매시장,380401,진주중앙청과,38040102,0000000752,20150910080000,1,경매,...,121100,특,11,2,계통,740010,경북 김천시 모암동,3703,경북김천,50


In [31]:
df = df.sort_values(by=['DELNG_DE'], axis=0)
df

,ROW_NUM,DELNG_DE,PBLMNG_WHSAL_MRKT_NM,PBLMNG_WHSAL_MRKT_CD,CPR_NM,CPR_CD,RISENO,BIDTIME,AUC_SE_CODE,AUC_SE_NM,...,STNDRD_CD,GRAD,GRAD_CD,SHIPMNT_SE_CODE,SHIPMNT_SE_NM,SANJI_CD,SANJI_NM,CPR_USE_SANJI_CD,CPR_USE_SANJI_NM,DELNG_QY
693846,915,20150701,구리도매시장,311201,인터넷청과,31120106,1275,,2,정가수의,...,121101,특,11,1,협동,471854,,471854,,300
693381,450,20150701,대구북부도매,220001,청과상장예외,22000111,0000001011,150701,2,정가수의,...,120000,보통,13,3,개별,770000,경북 영천시,770000,경북 영천시,100
693382,451,20150701,대구북부도매,220001,청과상장예외,22000111,0000001000,150701,2,정가수의,...,120000,없음,10,3,개별,718000,경북 칠곡군,718000,경북 칠곡군,100
693383,452,20150701,대구북부도매,220001,청과상장예외,22000111,0000001001,150701,2,정가수의,...,120000,없음,10,3,개별,718000,경북 칠곡군,718000,경북 칠곡군,100
693384,453,20150701,대구북부도매,220001,청과상장예외,22000111,0000001002,150701,2,정가수의,...,120000,없음,10,3,개별,718000,경북 칠곡군,718000,경북 칠곡군,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1587360,445,20210916,서울가락도매,110001,서울청과,11000101,1,20210916,1,경매,...,120700,특,11,,,250000,강원 홍천군,,,60
1587359,444,20210916,서울가락도매,110001,서울청과,11000101,1,20210916,1,경매,...,120700,특,11,,,250000,강원 홍천군,,,60
1587358,443,20210916,서울가락도매,110001,중앙청과,11000103,1,20210916,1,경매,...,120700,특,11,,,250000,강원 홍천군,,,44
1587366,451,20210916,광주서부도매,240004,호남청과,24000403,00001,20210916053953,1,경매,...,120100,특,11,1,협동,250809,강원 홍천군 홍천읍,3231,강원 홍천군,11


In [32]:
df.to_csv('./도매시장가격20150701_20210916.csv',encoding='cp949')

In [63]:
# ~16일 + 9월17~30일 data 추가생성
df_past = pd.read_csv("./도매시장가격20150701_20210916.csv", encoding="cp949")
df_past = df_past.drop('Unnamed: 0', axis=1)

path = './data/도매시장경매가격/'
file_list = os.listdir(path)

n_file = []
dates = range(17,31)

C:\Users\123gu\anaconda3\envs\DataEn\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (11,21,23) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


[]

In [64]:
for file in file_list:
    file = file.rstrip()
    for date in dates:
        if file.startswith('202109{}'.format(date)):
            n_file.append(file)
n_file

['20210917_1001_2000.json',
 '20210917_1_1000.json',
 '20210918_1_1000.json',
 '20210919_1_1000.json',
 '20210920_1_1000.json',
 '20210921_1_1000.json',
 '20210922_1_1000.json',
 '20210923_1_1000.json',
 '20210924_1001_2000.json',
 '20210924_1_1000.json',
 '20210925_1_1000.json',
 '20210926_1_1000.json',
 '20210927_1001_2000.json',
 '20210927_1_1000.json',
 '20210928_1001_2000.json',
 '20210928_1_1000.json',
 '20210929_1001_2000.json',
 '20210929_1_1000.json',
 '20210930_1001_2000.json',
 '20210930_1_1000.json']

In [65]:
with open(path+n_file[0], "r", encoding='utf-8') as json_file:
    json_data = json.load(json_file)
    data = json.dumps(json_data, indent = '\t', ensure_ascii=False)
d = json.loads(data)   
if d['Grid_20180118000000000581_1']['row'] == None:
    print('holy')

df_base = pd.DataFrame(d['Grid_20180118000000000581_1']['row'])
df = pd.DataFrame(columns = df_base.columns)
df

,ROW_NUM,DELNG_DE,PBLMNG_WHSAL_MRKT_NM,PBLMNG_WHSAL_MRKT_CD,CPR_NM,CPR_CD,RISENO,BIDTIME,AUC_SE_CODE,AUC_SE_NM,...,STNDRD_CD,GRAD,GRAD_CD,SHIPMNT_SE_CODE,SHIPMNT_SE_NM,SANJI_CD,SANJI_NM,CPR_USE_SANJI_CD,CPR_USE_SANJI_NM,DELNG_QY


In [66]:
df = pd.DataFrame(columns = df_base.columns)

for i in range(0, len(n_file)):
    with open(path+n_file[i], "r", encoding='utf-8') as json_file:
        json_data = json.load(json_file)
        data = json.dumps(json_data, indent = '\t', ensure_ascii=False)
    d = json.loads(data)
    if d['Grid_20180118000000000581_1']['row'] == None:
        continue
    else:
        df = df.append(d['Grid_20180118000000000581_1']['row'], ignore_index=True)

df

,ROW_NUM,DELNG_DE,PBLMNG_WHSAL_MRKT_NM,PBLMNG_WHSAL_MRKT_CD,CPR_NM,CPR_CD,RISENO,BIDTIME,AUC_SE_CODE,AUC_SE_NM,...,STNDRD_CD,GRAD,GRAD_CD,SHIPMNT_SE_CODE,SHIPMNT_SE_NM,SANJI_CD,SANJI_NM,CPR_USE_SANJI_CD,CPR_USE_SANJI_NM,DELNG_QY
0,1001,20210917,순천도매시장,360301,남도청과,36030101,00001,20210916140918,1,경매,...,121100,특,11,3,개별,565900,전북 완주군 봉동읍,565900,전북 완주군 봉동읍,55
1,1002,20210917,광주서부도매,240004,두레청과,24000402,00001,20210917045236,1,경매,...,121100,특,11,3,개별,565920,전북 완주군 경천면,565-920,전북 완주군 경천면,130
2,1003,20210917,안양도매시장,310401,안양농산물(주),31040105,9,20210917023323,1,경매,...,121100,특,11,3,개별,579820,전북 부안군 계화면,579820,전북 부안군 계화면,100
3,1004,20210917,안양도매시장,310401,안양농산물(주),31040105,8,20210917023319,1,경매,...,121100,특,11,3,개별,579820,전북 부안군 계화면,579820,전북 부안군 계화면,100
4,1005,20210917,안양도매시장,310401,안양농산물(주),31040105,7,20210917023315,1,경매,...,121100,특,11,3,개별,579820,전북 부안군 계화면,579820,전북 부안군 계화면,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9637,996,20210930,부산반여도매,210009,동부청과,21000901,00001,20210930125857,2,정가수의,...,120100,상,12,3,개별,902000,부산광역시,902000,부산시,5
9638,997,20210930,부산반여도매,210009,동부청과,21000901,00001,20210930125857,2,정가수의,...,120100,상,12,3,개별,902000,부산광역시,902000,부산시,10
9639,998,20210930,부산반여도매,210009,동부청과,21000901,00001,20210930125857,2,정가수의,...,120100,상,12,3,개별,902000,부산광역시,902000,부산시,10
9640,999,20210930,부산반여도매,210009,동부청과,21000901,00001,20210930125857,2,정가수의,...,120100,상,12,3,개별,902000,부산광역시,902000,부산시,5


In [68]:
df_recent = pd.concat([df_past, df])
df_recent

,ROW_NUM,DELNG_DE,PBLMNG_WHSAL_MRKT_NM,PBLMNG_WHSAL_MRKT_CD,CPR_NM,CPR_CD,RISENO,BIDTIME,AUC_SE_CODE,AUC_SE_NM,...,STNDRD_CD,GRAD,GRAD_CD,SHIPMNT_SE_CODE,SHIPMNT_SE_NM,SANJI_CD,SANJI_NM,CPR_USE_SANJI_CD,CPR_USE_SANJI_NM,DELNG_QY
0,915,20150701,구리도매시장,311201,인터넷청과,31120106,1275,NaN,2,정가수의,...,121101,특,11,1.0,협동,471854,NaN,471854,NaN,300
1,450,20150701,대구북부도매,220001,청과상장예외,22000111,0000001011,150701.0,2,정가수의,...,120000,보통,13,3.0,개별,770000,경북 영천시,770000,경북 영천시,100
2,451,20150701,대구북부도매,220001,청과상장예외,22000111,0000001000,150701.0,2,정가수의,...,120000,없음,10,3.0,개별,718000,경북 칠곡군,718000,경북 칠곡군,100
3,452,20150701,대구북부도매,220001,청과상장예외,22000111,0000001001,150701.0,2,정가수의,...,120000,없음,10,3.0,개별,718000,경북 칠곡군,718000,경북 칠곡군,100
4,453,20150701,대구북부도매,220001,청과상장예외,22000111,0000001002,150701.0,2,정가수의,...,120000,없음,10,3.0,개별,718000,경북 칠곡군,718000,경북 칠곡군,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9637,996,20210930,부산반여도매,210009,동부청과,21000901,00001,20210930125857,2,정가수의,...,120100,상,12,3,개별,902000,부산광역시,902000,부산시,5
9638,997,20210930,부산반여도매,210009,동부청과,21000901,00001,20210930125857,2,정가수의,...,120100,상,12,3,개별,902000,부산광역시,902000,부산시,10
9639,998,20210930,부산반여도매,210009,동부청과,21000901,00001,20210930125857,2,정가수의,...,120100,상,12,3,개별,902000,부산광역시,902000,부산시,10
9640,999,20210930,부산반여도매,210009,동부청과,21000901,00001,20210930125857,2,정가수의,...,120100,상,12,3,개별,902000,부산광역시,902000,부산시,5


In [ ]:
df_recent.to_csv('./data/도매시장가격_20150701_20210930.csv', encoding='cp949')

과거데이터 수집대상기간 2015.07.01 ~ 2021.09.07  

최신데이터 수집 추가 2021.09.08~

### 전국 도매시장 일별 정산 경락가격 상세정보

In [238]:
start = datetime.strptime("20210831", "%Y%m%d")
end = datetime.strptime("20210907", "%Y%m%d")
dates = [(start + timedelta(days=x)).strftime("%Y%m%d") for x in range(0, 1+(end-start).days)]

['20210831',
 '20210901',
 '20210902',
 '20210903',
 '20210904',
 '20210905',
 '20210906',
 '20210907']

In [247]:
# import pymysql
import requests
# from bs4 import BeautifulSoup
from urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter
import time
import os
import json
import xmltodict
from tqdm.notebook import tqdm

s = requests.Session()
retries = Retry(total=5,
               backoff_factor=4, # 2, 4, 8, 16, 32
               status_forcelist=[500, 502, 503, 504])
headers= {
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:87.0) Gecko/20100101 Firefox/87.0'
}

def crawl(base_url, params, directory, name):
    today = datetime.now()
    res = s.get(base_url, headers=headers, params=params)
    if res.status_code == 200:
        result = json.loads(json.dumps(xmltodict.parse(res.content.decode('utf-8')), ensure_ascii=False))
        with open(directory + today.strftime("%Y%m%d%H%M%S") + name + ".json", "w", encoding='utf-8') as json_file:
            json.dump(result, json_file, ensure_ascii=False)
        return result
    else:
        return res.status_code
    
def crawl_json(base_url, params, directory, name):
    today = datetime.now()
    res = s.get(base_url, headers=headers, params=params)
    if res.status_code == 200:
        result = json.loads(res.content.decode('utf-8'))
        with open(directory + name + ".json", "w", encoding='utf-8') as json_file:
            json.dump(result, json_file, ensure_ascii=False)
        return result
    else:
        return res.status_code
keys = [
    '3316101bc570d80943c917dcaf175c59da913ddbb38519f9103d9dbc46d5aef7'
]

In [252]:
slash = [keys[0], 'json', 'Grid_20180118000000000581_1', 0, 0]

start_date_i = 0
start = datetime.strptime("20210908", "%Y%m%d")
end = datetime.strptime("20210908", "%Y%m%d")
dates = [(start + timedelta(days=x)).strftime("%Y%m%d") for x in range(0, 1+(end-start).days)]
# dates = ['20210901']
for date_i in tqdm(range(len(dates))):
    date = dates[date_i]
    if date_i < start_date_i:
        continue
    params = {
        'DELNG_DE': date,
        'PRDLST_CD': 1202,
    }
    time.sleep(0.3)
    slash[3] = str(1)
    slash[4] = str(1000)
    base_url = 'http://211.237.50.150:7080/openapi/' + '/'.join(slash)
    result = crawl_json(base_url, params, './Data/도매시장경매가격/', '_'.join([date, slash[3], slash[4]]))
    if type(result) == int:
        print(date + ' error')
    end = result['Grid_20180118000000000581_1']['totalCnt'] // 1000 + 1
    for i in tqdm(range(1, end), leave=False):
        time.sleep(0.3)
        slash[3] = str(i*1000 + 1)
        slash[4] = str((i+1)*1000)
        base_url = 'http://211.237.50.150:7080/openapi/' + '/'.join(slash)
        crawl_json(base_url, params, './Data/도매시장경매가격/', '_'.join([date, slash[3], slash[4]]))

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [121]:
# 데이터확인
file_path = "./data/도매시장경매가격/20210809_1_1000.json"
with open(file_path, "r", encoding='utf-8') as json_file:
    json_data = json.load(json_file)
    data = json.dumps(json_data, indent = '\t', ensure_ascii=False)
    print(data)

{
	"Grid_20180118000000000581_1": {
		"totalCnt": 1263,
		"startRow": 1,
		"endRow": 1000,
		"result": {
			"code": "INFO-000",
			"message": "정상 처리되었습니다."
		},
		"row": [
			{
				"ROW_NUM": 1,
				"DELNG_DE": "20210809",
				"PBLMNG_WHSAL_MRKT_NM": "서울가락도매",
				"PBLMNG_WHSAL_MRKT_CD": "110001",
				"CPR_NM": "서울청과",
				"CPR_CD": "11000101",
				"RISENO": "1",
				"BIDTIME": "20210809",
				"AUC_SE_CODE": "1",
				"AUC_SE_NM": "경매",
				"ORGNO": "210812015839379",
				"PRDLST_NM": "대파",
				"PRDLST_CD": "1202",
				"SPCIES_NM": "대파(일반)",
				"SPCIES_CD": "120201",
				"CPR_USE_PRDLST_NM": "",
				"CPR_USE_SPCIES_NM": "",
				"PRICE": 940,
				"DELNGBUNDLE_QY": 1,
				"STNDRD": "kg 비닐봉지 ",
				"STNDRD_CD": "120800",
				"GRAD": "특",
				"GRAD_CD": "11",
				"SHIPMNT_SE_CODE": "",
				"SHIPMNT_SE_NM": "",
				"SANJI_CD": "410000",
				"SANJI_NM": "경기도  고양시",
				"CPR_USE_SANJI_CD": "",
				"CPR_USE_SANJI_NM": "",
				"DELNG_QY": 500
			},
			{
				"ROW_NUM": 2,
				"DELNG_DE": "202108

## 농촌진흥청 국립농업과학원_농업기상 기본관측데이터 조회
### 농업기상 조회기간별 일 기본 관측데이터 조회

인증키 encoding  

운영계정 신청함(일일 트래픽 10000)
cRubis4JVspr1lTWwQzXYqKwYs4tRA7P5S1W7Vo%2Bi%2FxenmhOzGzusFs3XSInzRFLAW0j7IdxSIWywgQL3e9o5Q%3D%3D

Bai%2FvTgIbiRsY5w5O7aVL%2BLHplODFxh0r0aafEu2mjWuRQY0s70R3Kzl3gszDRRXDL128%2BYnn4wfcHeZ3jD0tQ%3D%3D

Cmgie9fneM4cfj2ViQV5T2SU9poQGtlmBhpVAztPgY4PTtArNthKa4VmCTKxNQNnWy4Kz3PKw6qwmRbfu2sXzw%3D%3D



In [213]:
certf = ['cRubis4JVspr1lTWwQzXYqKwYs4tRA7P5S1W7Vo%2Bi%2FxenmhOzGzusFs3XSInzRFLAW0j7IdxSIWywgQL3e9o5Q%3D%3D','Bai%2FvTgIbiRsY5w5O7aVL%2BLHplODFxh0r0aafEu2mjWuRQY0s70R3Kzl3gszDRRXDL128%2BYnn4wfcHeZ3jD0tQ%3D%3D','Cmgie9fneM4cfj2ViQV5T2SU9poQGtlmBhpVAztPgY4PTtArNthKa4VmCTKxNQNnWy4Kz3PKw6qwmRbfu2sXzw%3D%3D']

spot = ['신안군','진도군','영광군','완주군','부산시','청주시','아산시','구미시','평창군','남양주','용인시,','양주시','김포시','김제시', '의성군','무안군','강진군','장성군','김해시','세종시','예산군','횡성군','광주시','이천시','고양시','안산시']

spot_code = ['535812A001','539823A001','513842A001','055365F016','618803A001','363844A001','336812A001','039102B003','232803A001','472830A001','449872A001','482841A001','415743A001','576040A001','037339B009','534833E001','059223A001','515804A001','621100A001','339814A001','340861A001','225874A001','464030A001','467800A001','412040A002','425320A001']

start = ['2015-07-01','2016-01-01','2017-01-01','2018-01-01','2019-01-01','2020-01-01','2021-01-01']
end = ['2015-12-31','2016-12-31','2017-12-31','2018-12-31','2019-12-31','2020-12-31','2021-09-07']

page = ['1','2','3','4']


In [217]:
for i in range(0, len(start)):
    for j in range(0, len(spot_code)):
        for k in range(0, len(page)):
            base_url = 'http://apis.data.go.kr/1390802/AgriWeather/WeatherObsrInfo/GnrlWeather/getWeatherTermDayList?'
            service = 'serviceKey={}'.format(certf[0])
            page_no = '&Page_No={}&Page_Size=100'.format(page[k]) 
            begin = '&begin_Date={}'.format(start[i])
            finish = '&end_Date={}'.format(end[i])
            obs = '&obsr_Spot_Code={}'.format(spot_code[j])

            url = base_url + service +page_no + begin + finish + obs
            response = requests.get(url)

            # ensure_ascii = False : 파일이 저장될때 한글이 유니코드 형태로 저장되지 않도록 함
            if response.status_code == 200:
                xd = xmltodict.parse(response.text) # XML 형식 데이터를 dict 형식으로 바꾼다
                xdj = json.dumps(xd, ensure_ascii=False) # dict형식의 데이터를 json 형식으로 변환
                xdjd = json.loads(xdj) # json형식의 데이터를 dict형식으로 변환

                file_path = "./data/일별농업기상/weather{}_{}_{}_{}.json".format(spot[j],start[i],end[i],page[k])
                with open(file_path, 'w', encoding='utf-8') as outfile:
                    json.dump(xdjd, outfile, indent='\t', ensure_ascii=False)
                print(spot[j],start[i],end[i],page[k])
            else:
                print(response.status_code)


김제시 2015-07-01 2015-12-31 1
김제시 2015-07-01 2015-12-31 2
김제시 2015-07-01 2015-12-31 3
김제시 2015-07-01 2015-12-31 4
의성군 2015-07-01 2015-12-31 1
의성군 2015-07-01 2015-12-31 2
의성군 2015-07-01 2015-12-31 3
의성군 2015-07-01 2015-12-31 4
무안군 2015-07-01 2015-12-31 1
무안군 2015-07-01 2015-12-31 2
무안군 2015-07-01 2015-12-31 3
무안군 2015-07-01 2015-12-31 4
강진군 2015-07-01 2015-12-31 1
강진군 2015-07-01 2015-12-31 2
강진군 2015-07-01 2015-12-31 3
강진군 2015-07-01 2015-12-31 4
장성군 2015-07-01 2015-12-31 1
장성군 2015-07-01 2015-12-31 2
장성군 2015-07-01 2015-12-31 3
장성군 2015-07-01 2015-12-31 4
김해시 2015-07-01 2015-12-31 1
김해시 2015-07-01 2015-12-31 2
김해시 2015-07-01 2015-12-31 3
김해시 2015-07-01 2015-12-31 4
세종시 2015-07-01 2015-12-31 1
세종시 2015-07-01 2015-12-31 2
세종시 2015-07-01 2015-12-31 3
세종시 2015-07-01 2015-12-31 4
예산군 2015-07-01 2015-12-31 1
예산군 2015-07-01 2015-12-31 2
예산군 2015-07-01 2015-12-31 3
예산군 2015-07-01 2015-12-31 4
횡성군 2015-07-01 2015-12-31 1
횡성군 2015-07-01 2015-12-31 2
횡성군 2015-07-01 2015-12-31 3
횡성군 2015-07-01 2015-

In [147]:
# 데이터확인
file_path = "./data/일별농업기상/weather남양주_2015-07-01_2015-12-31_1.json"
with open(file_path, "r", encoding='utf-8') as json_file:
    json_data = json.load(json_file)
    data = json.dumps(json_data, indent = '\t', ensure_ascii=False)
    print(data)

{
	"response": {
		"header": {
			"result_Code": "200",
			"result_Msg": "OK"
		},
		"body": {
			"rcdcnt": "100",
			"page_No": "1",
			"total_Count": "184",
			"items": {
				"item": [
					{
						"no": "1",
						"stn_Code": "472830A001",
						"stn_Name": "남양주 진건읍",
						"date": "2015-07-01",
						"temp": "24",
						"max_Temp": "29.3",
						"min_Temp": "19.8",
						"hum": "72.9",
						"widdir": "232.4",
						"wind": "1.3",
						"rain": "0.5",
						"sun_Time": "626",
						"sun_Qy": "21.9",
						"condens_Time": "0",
						"gr_Temp": "0",
						"soil_Temp": "33.6",
						"soil_Wt": null
					},
					{
						"no": "2",
						"stn_Code": "472830A001",
						"stn_Name": "남양주 진건읍",
						"date": "2015-07-02",
						"temp": "23.7",
						"max_Temp": "29.7",
						"min_Temp": "18.1",
						"hum": "67.3",
						"widdir": "202.9",
						"wind": "1.1",
						"rain": "0",
						"sun_Time": "618",
						"sun_Qy": "18.6",
						"condens_Time": "0",
						"gr_Temp": "0",
						"soil_Te

In [103]:
d = json.loads(data)
d['response']['body']

{'rcdcnt': '100', 'page_No': '3', 'total_Count': '184', 'items': None}

## 유통정보(시장도매인거래) - 품목별등급별가격

20150701 ~ 20210914

In [79]:
start = datetime.strptime("20180729", "%Y%m%d")
end = datetime.strptime("20210914", "%Y%m%d")

# 당일
dates = [(start + timedelta(days=x)).strftime("%Y%m%d") for x in range(0, 1+(end-start).days)]

# 하루전
dates_p = []
for i in range(0, 1+(end-start).days):
    pre = [(start + timedelta(days=x)) for x in range(0, 1+(end-start).days)][i] - timedelta(1)
    dates_p.append(pre.strftime('%Y%m%d'))

# 일주전
dates_s = []
for i in range(0, 1+(end-start).days):
    sev = [(start + timedelta(days=x)) for x in range(0, 1+(end-start).days)][i] - timedelta(7)
    dates_s.append(sev.strftime('%Y%m%d'))


In [80]:
for i in range(0,len(dates)):
    base_url = 'http://www.garak.co.kr/publicdata/dataOpen.do?id=3043&passwd=multi1234!&dataid=data37&pagesize=10&pageidx=1&portal.templet=false'
    date = '&s_date={}'.format(dates[i])
    date_p = '&s_date_p={}'.format(dates_p[i])
    date_s = '&s_date_p7={}'.format(dates_s[i])
    product = '&s_pum_nm=2&s_pummok={}'.format('대파')

    url = base_url + date + date_p + date_s + product
    response = requests.get(url)

    # ensure_ascii = False : 파일이 저장될때 한글이 유니코드 형태로 저장되지 않도록 함
    if response.status_code == 200:
        xd = xmltodict.parse(response.text) # XML 형식 데이터를 dict 형식으로 바꾼다
        xdj = json.dumps(xd, ensure_ascii=False) # dict형식의 데이터를 json 형식으로 변환
        xdjd = json.loads(xdj) # json형식의 데이터를 dict형식으로 변환

        file_path = "./data/시장도매인가격/시장도매{}.json".format(dates[i])
        with open(file_path, 'w', encoding='utf-8') as outfile:
            json.dump(xdjd, outfile, indent='\t', ensure_ascii=False)
        print(dates[i])
    else:
        print(response.status_code)

20180729
20180730
20180731
20180801
20180802
20180803
20180804
20180805
20180806
20180807
20180808
20180809
20180810
20180811
20180812
20180813
20180814
20180815
20180816
20180817
20180818
20180819
20180820
20180821
20180822
20180823
20180824
20180825
20180826
20180827
20180828
20180829
20180830
20180831
20180901
20180902
20180903
20180904
20180905
20180906
20180907
20180908
20180909
20180910
20180911
20180912
20180913
20180914
20180915
20180916
20180917
20180918
20180919
20180920
20180921
20180922
20180923
20180924
20180925
20180926
20180927
20180928
20180929
20180930
20181001
20181002
20181003
20181004
20181005
20181006
20181007
20181008
20181009
20181010
20181011
20181012
20181013
20181014
20181015
20181016
20181017
20181018
20181019
20181020
20181021
20181022
20181023
20181024
20181025
20181026
20181027
20181028
20181029
20181030
20181031
20181101
20181102
20181103
20181104
20181105
20181106
20181107
20181108
20181109
20181110
20181111
20181112
20181113
20181114
20181115
20181116
2

20210125
20210126
20210127
20210128
20210129
20210130
20210131
20210201
20210202
20210203
20210204
20210205
20210206
20210207
20210208
20210209
20210210
20210211
20210212
20210213
20210214
20210215
20210216
20210217
20210218
20210219
20210220
20210221
20210222
20210223
20210224
20210225
20210226
20210227
20210228
20210301
20210302
20210303
20210304
20210305
20210306
20210307
20210308
20210309
20210310
20210311
20210312
20210313
20210314
20210315
20210316
20210317
20210318
20210319
20210320
20210321
20210322
20210323
20210324
20210325
20210326
20210327
20210328
20210329
20210330
20210331
20210401
20210402
20210403
20210404
20210405
20210406
20210407
20210408
20210409
20210410
20210411
20210412
20210413
20210414
20210415
20210416
20210417
20210418
20210419
20210420
20210421
20210422
20210423
20210424
20210425
20210426
20210427
20210428
20210429
20210430
20210501
20210502
20210503
20210504
20210505
20210506
20210507
20210508
20210509
20210510
20210511
20210512
20210513
20210514
20210515
2

In [81]:
# 데이터확인
file_path = "./data/시장도매인가격/시장도매20210914.json"
with open(file_path, "r", encoding='utf-8') as json_file:
    json_data = json.load(json_file)
    data = json.dumps(json_data, indent = '\t', ensure_ascii=False)
    print(data)

{
	"lists": {
		"list_total_count": "4",
		"list": [
			{
				"PUM_NM": "대파(일반)",
				"G_NAME": "특",
				"U_NAME": "1 kg단",
				"MI_P": "1,800",
				"MA_P": "2,200",
				"AV_P": "1,957",
				"PAV_P": "2,003",
				"FLUC": "-46",
				"D_MARK": "98.0",
				"J_7": "1,933",
				"J_7_MARK": "101.2"
			},
			{
				"PUM_NM": "대파(일반)",
				"G_NAME": "상",
				"U_NAME": "1 kg단",
				"MI_P": "1,500",
				"MA_P": "1,800",
				"AV_P": "1,595",
				"PAV_P": "1,580",
				"FLUC": "15",
				"D_MARK": "101.0",
				"J_7": "1,599",
				"J_7_MARK": "99.8"
			},
			{
				"PUM_NM": "대파(일반)",
				"G_NAME": "보통",
				"U_NAME": "1 kg단",
				"MI_P": "1,300",
				"MA_P": "1,500",
				"AV_P": "1,435",
				"PAV_P": "1,422",
				"FLUC": "13",
				"D_MARK": "101.0",
				"J_7": "1,357",
				"J_7_MARK": "105.8"
			},
			{
				"PUM_NM": "대파(일반)",
				"G_NAME": "하",
				"U_NAME": "1 kg단",
				"MI_P": "500",
				"MA_P": "1,300",
				"AV_P": "1,013",
				"PAV_P": "1,106",
				"FLUC": "-93",
				"D_MARK": "92.0",
				"J_7": 

## 여기부터 사용안함

## 수출입은행 현재환율
인증키  
xCiBdwrujOWcCNaQcBz78bpI3oykq4Kh

&searchdate=20180102

190528 부터 result 4뜸

In [3]:
date_index = pd.date_range(start = '20190528', end = '20210831')
date = date_index.strftime("%Y%m%d").tolist()

In [4]:
len(date)

827

In [19]:
for i in range(0, len(date)):
    file_path = "./data/수출입은행현재환율/ExchangeRate{}.json".format(date[i])
    url = 'https://www.koreaexim.go.kr/site/program/financial/exchangeJSON?authkey=xCiBdwrujOWcCNaQcBz78bpI3oykq4Kh&searchdate={}&data=AP01'.format(date[i])
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        with open(file_path, 'w', encoding='utf-8') as outfile:
            json.dump(data, outfile, indent='\t')
        print(date[i])
  


20190528
20190529
20190530
20190531
20190601
20190602
20190603
20190604
20190605
20190606
20190607
20190608
20190609
20190610
20190611
20190612
20190613
20190614
20190615
20190616
20190617
20190618
20190619
20190620
20190621
20190622
20190623
20190624
20190625
20190626
20190627
20190628
20190629
20190630
20190701
20190702
20190703
20190704
20190705
20190706
20190707
20190708
20190709
20190710
20190711
20190712
20190713
20190714
20190715
20190716
20190717
20190718
20190719
20190720
20190721
20190722
20190723
20190724
20190725
20190726
20190727
20190728
20190729
20190730
20190731
20190801
20190802
20190803
20190804
20190805
20190806
20190807
20190808
20190809
20190810
20190811
20190812
20190813
20190814
20190815
20190816
20190817
20190818
20190819
20190820
20190821
20190822
20190823
20190824
20190825
20190826
20190827
20190828
20190829
20190830
20190831
20190901
20190902
20190903
20190904
20190905
20190906
20190907
20190908
20190909
20190910
20190911
20190912
20190913
20190914
20190915
2

데이터 확인

In [25]:
file_path = "./data/수출입은행현재환율/ExchangeRate20210831.json"
with open(file_path, "r", encoding='utf-8') as json_file:
    json_data = json.load(json_file)
    print(json.dumps(json_data, indent = '\t', ensure_ascii=False))

[
	{
		"result": 1,
		"cur_unit": "AED",
		"ttb": "313.83",
		"tts": "320.17",
		"deal_bas_r": "317",
		"bkpr": "317",
		"yy_efee_r": "0",
		"ten_dd_efee_r": "0",
		"kftc_bkpr": "317",
		"kftc_deal_bas_r": "317",
		"cur_nm": "아랍에미리트 디르함"
	},
	{
		"result": 1,
		"cur_unit": "AUD",
		"ttb": "840.87",
		"tts": "857.86",
		"deal_bas_r": "849.37",
		"bkpr": "849",
		"yy_efee_r": "0",
		"ten_dd_efee_r": "0",
		"kftc_bkpr": "849",
		"kftc_deal_bas_r": "849.37",
		"cur_nm": "호주 달러"
	},
	{
		"result": 1,
		"cur_unit": "BHD",
		"ttb": "3,057.7",
		"tts": "3,119.47",
		"deal_bas_r": "3,088.59",
		"bkpr": "3,088",
		"yy_efee_r": "0",
		"ten_dd_efee_r": "0",
		"kftc_bkpr": "3,088",
		"kftc_deal_bas_r": "3,088.59",
		"cur_nm": "바레인 디나르"
	},
	{
		"result": 1,
		"cur_unit": "BND",
		"ttb": "856.84",
		"tts": "874.15",
		"deal_bas_r": "865.5",
		"bkpr": "865",
		"yy_efee_r": "0",
		"ten_dd_efee_r": "0",
		"kftc_bkpr": "865",
		"kftc_deal_bas_r": "865.5",
		"cur_nm": "브루나이 달러"
	},
	{
		"result": 1,
		"c

In [22]:
response.json()[0]['cur_unit']

'AED'

### 팜맵기반 농업 기상 조회 서비스

인증키 encode :  
cRubis4JVspr1lTWwQzXYqKwYs4tRA7P5S1W7Vo%2Bi%2FxenmhOzGzusFs3XSInzRFLAW0j7IdxSIWywgQL3e9o5Q%3D%3D


좌표계 : EPSG5179

#### 일자별 조회

샘플 좌표 입력 시
210423 까지 있음

In [ ]:
date1_index = pd.date_range(start = '20160901', end = '20190528')
date1 = date1_index.strftime("%Y%m%d").tolist()

date2_index = pd.date_range(start = '20190529', end = '20210831')
date2 = date2_index.strftime("%Y%m%d").tolist()

In [ ]:
# 좌표변환

epsg5179 = from_string("+proj=tmerc +lat_0=38 +lon_0=127.5 +k=0.9996 +x_0=1000000 +y_0=2000000 +ellps=GRS80 +units=m +no_defs")



In [47]:
url = 'http://apis.data.go.kr/B552895/rest/farmmap/getFarmmapAgricultureWeatherService/getCoordinateBasedDayFarmimgWeatherInfo?serviceKey=cRubis4JVspr1lTWwQzXYqKwYs4tRA7P5S1W7Vo%2Bi%2FxenmhOzGzusFs3XSInzRFLAW0j7IdxSIWywgQL3e9o5Q%3D%3D&numOfRows=10&pageNo=1&type=JSON&positionX=882327.8982&positionY=1674445.3078&month=202004&yearCount=1'
response = requests.get(url)



In [48]:
response.status_code

200

In [50]:
response.text

'<?xml version="1.0" encoding="UTF-8" standalone="yes"?>\n<response>\n    <header>\n        <resultCode>3</resultCode>\n        <resultMsg>NODATA_ERROR</resultMsg>\n    </header>\n</response>\n'

In [ ]:
# date1
for i in range(0, len(date1)):
    for d in range(0,len(time))
    file_path = "./data/초단기실황/UltraShortLive{}_{}.json".format(date1[i],time[d])
    url = 'https://www.koreaexim.go.kr/site/program/financial/exchangeJSON?authkey=xCiBdwrujOWcCNaQcBz78bpI3oykq4Kh&searchdate={}&data=AP01'.format(date[i])
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        with open(file_path, 'w', encoding='utf-8') as outfile:
            json.dump(data, outfile, indent='\t')
        print(date[i])

#### 시간별 조회

In [ ]:
url = 'http://apis.data.go.kr/B552895/rest/farmmap/getFarmmapAgricultureWeatherService/getCoordinateBasedHourFarmimgWeatherInfo?ServiceKey=서비스키&serviceKey=인증키(URL Encode)&numOfRows=10&pageNo=1&type=xml&positionX=1121355.550488932&positionY=1976977.084509703&date=20180524&yearCount=3'


## 전국 도매시장 일별 정산 경락가격 상세정보

농림축산식품 공공데이터 포털  
인증키  
9b74606381cc008f6394900c067351bf88968e16694803ed1fe3f98d230e958a

전국 도매시장 일별 정산 경락가격 상세정보(법인사용코드포함) OpenApi  
192.168.3.3

시장이름 : []  
품목이름 : []  
기간 : 

In [ ]:
slash = [keys[0], 'json', 'Grid_20180118000000000581_1', 0, 0]

start_date_i = 0
start = datetime.datetime.strptime("20150701", "%Y%m%d")
end = datetime.datetime.strptime("20210831", "%Y%m%d")
dates = [(start + datetime.timedelta(days=x)).strftime("%Y%m%d") for x in range(0, (end-start).days)]
# dates = ['20210901']
for date_i in tqdm(range(len(dates))):
    date = dates[date_i]
    if date_i < start_date_i:
        continue
    params = {
        'DELNG_DE': date,
        'PRDLST_CD': 1202,
    }
    time.sleep(0.3)
    slash[3] = str(1)
    slash[4] = str(1000)
    base_url = 'http://211.237.50.150:7080/openapi/' + '/'.join(slash)
    result = crawl_json(base_url, params, '../Data/Wholesale/', '_'.join([date, slash[3], slash[4]]))
    if type(result) == int:
        print(date + ' error')
    end = result['Grid_20180118000000000581_1']['totalCnt'] // 1000 + 1
    for i in tqdm(range(1, end), leave=False):
        time.sleep(0.3)
        slash[3] = str(i*1000 + 1)
        slash[4] = str((i+1)*1000)
        base_url = 'http://211.237.50.150:7080/openapi/' + '/'.join(slash)
        crawl_json(base_url, params, '../Data/Wholesale/', '_'.join([date, slash[3], slash[4]]))


In [12]:
# 일정 기간의 날짜 생성
date_index = pd.date_range(start = '20210830', end = '20210901')
date = date_index.strftime("%Y%m%d").tolist()

In [ ]:
table = pd.DataFrame( columns=('ROW_NUM','DELNG_DE','SBID_TIME','WHSAL_MRKT_NEW_CODE','WHSAL_MRKT_NEW_NM','CPR_INSTT_NEW_CODE','INSTT_NEW_NM','CATGORY_NEW_CODE','CATGORY_NEW_NM','STD_PRDLST_NEW_CODE','STD_PRDLST_NEW_NM','STD_SPCIES_NEW_CODE','STD_SPCIES_NEW_NM','DELNGBUNDLE_QY','STD_UNIT_NEW_CODE','STD_UNIT_NEW_NM','SBID_PRIC','STD_MTC_NEW_CODE','STD_MTC_NEW_NM','DELNG_QY'))

for d in range(0,len(date)):
    for j in range(10):
        rows_1 = 1000*j + 1
        rows_2 = 1000*j + 1000
        url = 'http://211.237.50.150:7080/openapi/9b74606381cc008f6394900c067351bf88968e16694803ed1fe3f98d230e958a/xml/Grid_20160308000000000346_1/{}/{}?DELNG_DE={}&WHSAL_MRKT_NEW_NM={}'.format(rows_1,rows_2,d,'가락동농수산물시장')
        if response.status_code == 200:
            xd = xmltodict.parse(response.text) # XML 형식 데이터를 dict 형식으로 바꾼다
            xdj = json.dumps(xd, ensure_ascii=False) # dict형식의 데이터를 json 형식으로 변환
            xdjd = json.loads(xdj) # json형식의 데이터를 dict형식으로 변환
            
        for i in range(len(data)):
            table.loc[len(table)] = [data[i]['ROW_NUM'],data[i]['DELNG_DE'],data[i]['SBID_TIME'],data[i]['WHSAL_MRKT_NEW_CODE'],data[i]['WHSAL_MRKT_NEW_NM'],data[i]['CPR_INSTT_NEW_CODE'],data[i]['INSTT_NEW_NM'],data[i]['CATGORY_NEW_CODE'],data[i]['CATGORY_NEW_NM'],data[i]['STD_PRDLST_NEW_CODE'],data[i]['STD_PRDLST_NEW_NM'],data[i]['STD_SPCIES_NEW_CODE'],data[i]['STD_SPCIES_NEW_NM'],data[i]['DELNGBUNDLE_QY'],data[i]['STD_UNIT_NEW_CODE'],data[i]['STD_UNIT_NEW_NM'],data[i]['SBID_PRIC'],data[i]['STD_MTC_NEW_CODE'],data[i]['STD_MTC_NEW_NM'],data[i]['DELNG_QY']]



In [ ]:
table = pd.DataFrame( columns=('ROW_NUM','DELNG_DE','SBID_TIME','WHSAL_MRKT_NEW_CODE','WHSAL_MRKT_NEW_NM','CPR_INSTT_NEW_CODE','INSTT_NEW_NM','CATGORY_NEW_CODE','CATGORY_NEW_NM','STD_PRDLST_NEW_CODE','STD_PRDLST_NEW_NM','STD_SPCIES_NEW_CODE','STD_SPCIES_NEW_NM','DELNGBUNDLE_QY','STD_UNIT_NEW_CODE','STD_UNIT_NEW_NM','SBID_PRIC','STD_MTC_NEW_CODE','STD_MTC_NEW_NM','DELNG_QY'))

for d in range(0,len(date)):
    for j in range(10):
        rows_1 = 1000*j + 1
        rows_2 = 1000*j + 1000
        url = 'http://211.237.50.150:7080/openapi/9b74606381cc008f6394900c067351bf88968e16694803ed1fe3f98d230e958a/xml/Grid_20160308000000000346_1/{}/{}?DELNG_DE={}&WHSAL_MRKT_NEW_NM={}'.format(rows_1,rows_2,d,'가락동농수산물시장')
        if response.status_code == 200:
            xd = xmltodict.parse(response.text) # XML 형식 데이터를 dict 형식으로 바꾼다
            xdj = json.dumps(xd, ensure_ascii=False) # dict형식의 데이터를 json 형식으로 변환
            xdjd = json.loads(xdj) # json형식의 데이터를 dict형식으로 변환
            
        for i in range(len(data)):
            table.loc[len(table)] = [data[i]['ROW_NUM'],data[i]['DELNG_DE'],data[i]['SBID_TIME'],data[i]['WHSAL_MRKT_NEW_CODE'],data[i]['WHSAL_MRKT_NEW_NM'],data[i]['CPR_INSTT_NEW_CODE'],data[i]['INSTT_NEW_NM'],data[i]['CATGORY_NEW_CODE'],data[i]['CATGORY_NEW_NM'],data[i]['STD_PRDLST_NEW_CODE'],data[i]['STD_PRDLST_NEW_NM'],data[i]['STD_SPCIES_NEW_CODE'],data[i]['STD_SPCIES_NEW_NM'],data[i]['DELNGBUNDLE_QY'],data[i]['STD_UNIT_NEW_CODE'],data[i]['STD_UNIT_NEW_NM'],data[i]['SBID_PRIC'],data[i]['STD_MTC_NEW_CODE'],data[i]['STD_MTC_NEW_NM'],data[i]['DELNG_QY']]



KeyboardInterrupt: 

In [67]:
table.to_csv('./실시간도매가격수집.csv', encoding='CP949')

In [54]:
url = 'http://211.237.50.150:7080/openapi/9b74606381cc008f6394900c067351bf88968e16694803ed1fe3f98d230e958a/xml/Grid_20160308000000000346_1/1/1000?DELNG_DE=20210824&WHSAL_MRKT_NEW_NM=가락동농수산물시장'
response = requests.get(url)

if response.status_code == 200:
    xd = xmltodict.parse(response.text) # XML 형식 데이터를 dict 형식으로 바꾼다
    xdj = json.dumps(xd, ensure_ascii=False) # dict형식의 데이터를 json 형식으로 변환
    xdjd = json.loads(xdj) # json형식의 데이터를 dict형식으로 변환

xdjd['Grid_20160308000000000346_1']['row']

[{'ROW_NUM': '1',
  'DELNG_DE': '20210824',
  'SBID_TIME': '20210823211502',
  'WHSAL_MRKT_NEW_CODE': '1005601',
  'WHSAL_MRKT_NEW_NM': '가락동농수산물시장',
  'CPR_INSTT_NEW_CODE': '6005601',
  'INSTT_NEW_NM': '서울청과',
  'CATGORY_NEW_CODE': '10',
  'CATGORY_NEW_NM': '엽경채류',
  'STD_PRDLST_NEW_CODE': '1011',
  'STD_PRDLST_NEW_NM': '깻잎',
  'STD_SPCIES_NEW_CODE': '101199',
  'STD_SPCIES_NEW_NM': '기타',
  'DELNGBUNDLE_QY': '2',
  'STD_UNIT_NEW_CODE': '12',
  'STD_UNIT_NEW_NM': 'kg',
  'SBID_PRIC': '16500',
  'STD_MTC_NEW_CODE': '32700',
  'STD_MTC_NEW_NM': '충청남도 금산군',
  'DELNG_QY': '10'},
 {'ROW_NUM': '2',
  'DELNG_DE': '20210824',
  'SBID_TIME': '20210823232748',
  'WHSAL_MRKT_NEW_CODE': '1005601',
  'WHSAL_MRKT_NEW_NM': '가락동농수산물시장',
  'CPR_INSTT_NEW_CODE': '6005601',
  'INSTT_NEW_NM': '서울청과',
  'CATGORY_NEW_CODE': '12',
  'CATGORY_NEW_NM': '조미채소류',
  'STD_PRDLST_NEW_CODE': '1205',
  'STD_PRDLST_NEW_NM': '고추',
  'STD_SPCIES_NEW_CODE': '120510',
  'STD_SPCIES_NEW_NM': '오이맛고추',
  'DELNGBUNDLE_QY': '10

In [57]:
url = 'http://211.237.50.150:7080/openapi/9b74606381cc008f6394900c067351bf88968e16694803ed1fe3f98d230e958a/xml/Grid_20160308000000000346_1/1001/2000?DELNG_DE=20210824&WHSAL_MRKT_NEW_NM=가락동농수산물시장'
response = requests.get(url)

if response.status_code == 200:
    xd = xmltodict.parse(response.text) # XML 형식 데이터를 dict 형식으로 바꾼다
    xdj = json.dumps(xd, ensure_ascii=False) # dict형식의 데이터를 json 형식으로 변환
    xdjd = json.loads(xdj) # json형식의 데이터를 dict형식으로 변환

data = xdjd['Grid_20160308000000000346_1']['row']
data

[{'ROW_NUM': '1001',
  'DELNG_DE': '20210824',
  'SBID_TIME': '20210823222953',
  'WHSAL_MRKT_NEW_CODE': '1005601',
  'WHSAL_MRKT_NEW_NM': '가락동농수산물시장',
  'CPR_INSTT_NEW_CODE': '6005601',
  'INSTT_NEW_NM': '서울청과',
  'CATGORY_NEW_CODE': '17',
  'CATGORY_NEW_NM': '버섯류',
  'STD_PRDLST_NEW_CODE': '1711',
  'STD_PRDLST_NEW_NM': '새송이',
  'STD_SPCIES_NEW_CODE': '171101',
  'STD_SPCIES_NEW_NM': '새송이버섯',
  'DELNGBUNDLE_QY': '4',
  'STD_UNIT_NEW_CODE': '12',
  'STD_UNIT_NEW_NM': 'kg',
  'SBID_PRIC': '11500',
  'STD_MTC_NEW_CODE': '39600',
  'STD_MTC_NEW_NM': '경상북도 김천시',
  'DELNG_QY': '70'},
 {'ROW_NUM': '1002',
  'DELNG_DE': '20210824',
  'SBID_TIME': '20210823223340',
  'WHSAL_MRKT_NEW_CODE': '1005601',
  'WHSAL_MRKT_NEW_NM': '가락동농수산물시장',
  'CPR_INSTT_NEW_CODE': '6005601',
  'INSTT_NEW_NM': '서울청과',
  'CATGORY_NEW_CODE': '12',
  'CATGORY_NEW_NM': '조미채소류',
  'STD_PRDLST_NEW_CODE': '1205',
  'STD_PRDLST_NEW_NM': '고추',
  'STD_SPCIES_NEW_CODE': '120541',
  'STD_SPCIES_NEW_NM': '꽈리고추',
  'DELNGBUNDLE_

In [59]:
len(data)

1000

In [61]:
data[0]['ROW_NUM']

'1001'

In [63]:
table = pd.DataFrame( columns=('ROW_NUM','DELNG_DE','SBID_TIME','WHSAL_MRKT_NEW_CODE','WHSAL_MRKT_NEW_NM','CPR_INSTT_NEW_CODE','INSTT_NEW_NM','CATGORY_NEW_CODE','CATGORY_NEW_NM','STD_PRDLST_NEW_CODE','STD_PRDLST_NEW_NM','STD_SPCIES_NEW_CODE','STD_SPCIES_NEW_NM','DELNGBUNDLE_QY','STD_UNIT_NEW_CODE','STD_UNIT_NEW_NM','SBID_PRIC','STD_MTC_NEW_CODE','STD_MTC_NEW_NM','DELNG_QY'))

for i in range(len(data)):
    table.loc[len(table)] = [data[i]['ROW_NUM'],data[i]['DELNG_DE'],data[i]['SBID_TIME'],data[i]['WHSAL_MRKT_NEW_CODE'],data[i]['WHSAL_MRKT_NEW_NM'],data[i]['CPR_INSTT_NEW_CODE'],data[i]['INSTT_NEW_NM'],data[i]['CATGORY_NEW_CODE'],data[i]['CATGORY_NEW_NM'],data[i]['STD_PRDLST_NEW_CODE'],data[i]['STD_PRDLST_NEW_NM'],data[i]['STD_SPCIES_NEW_CODE'],data[i]['STD_SPCIES_NEW_NM'],data[i]['DELNGBUNDLE_QY'],data[i]['STD_UNIT_NEW_CODE'],data[i]['STD_UNIT_NEW_NM'],data[i]['SBID_PRIC'],data[i]['STD_MTC_NEW_CODE'],data[i]['STD_MTC_NEW_NM'],data[i]['DELNG_QY']]

In [64]:
table

,ROW_NUM,DELNG_DE,SBID_TIME,WHSAL_MRKT_NEW_CODE,WHSAL_MRKT_NEW_NM,CPR_INSTT_NEW_CODE,INSTT_NEW_NM,CATGORY_NEW_CODE,CATGORY_NEW_NM,STD_PRDLST_NEW_CODE,STD_PRDLST_NEW_NM,STD_SPCIES_NEW_CODE,STD_SPCIES_NEW_NM,DELNGBUNDLE_QY,STD_UNIT_NEW_CODE,STD_UNIT_NEW_NM,SBID_PRIC,STD_MTC_NEW_CODE,STD_MTC_NEW_NM,DELNG_QY
0,1001,20210824,20210823222953,1005601,가락동농수산물시장,6005601,서울청과,17,버섯류,1711,새송이,171101,새송이버섯,4,12,kg,11500,39600,경상북도 김천시,70
1,1002,20210824,20210823223340,1005601,가락동농수산물시장,6005601,서울청과,12,조미채소류,1205,고추,120541,꽈리고추,4,12,kg,26000,25300,강원도 평창군,17
2,1003,20210824,20210824023035,1005601,가락동농수산물시장,6005601,서울청과,06,과실류,0614,감귤,061405,하우스감귤,6,12,kg,32400,63500,제주특별자치도 서귀포시,10
3,1004,20210824,20210824023600,1005601,가락동농수산물시장,6005601,서울청과,06,과실류,0604,복숭아,060435,환타지아,10,12,kg,15000,38900,경상북도 영천시,21
4,1005,20210824,20210824032420,1005601,가락동농수산물시장,6005601,서울청과,06,과실류,0603,포도,060301,캠벨얼리,3,12,kg,12000,37100,경상북도 상주시,50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1996,20210824,20210824013129,1005601,가락동농수산물시장,6005601,서울청과,06,과실류,0603,포도,060394,크림슨(수입),8,12,kg,76000,00000,None,15
996,1997,20210824,20210823212345,1005601,가락동농수산물시장,6005601,서울청과,10,엽경채류,1011,깻잎,101101,깻잎(일반),3,12,kg,29000,32700,충청남도 금산군,6
997,1998,20210824,20210823222548,1005601,가락동농수산물시장,6005601,서울청과,09,과채류,0902,호박,090201,애호박,8,12,kg,11000,24200,강원도 춘천시,2
998,1999,20210824,20210823180539,1005601,가락동농수산물시장,6005601,서울청과,13,양채류,1329,라디초우,132910,라디초우(일반),4,12,kg,16000,27200,충청북도 제천시,19


In [ ]:
data_list = []

for name in file_list:
    temp_filename = folder_path + name
    temp_json = open(temp_filename, 'r').readline()
    temp_json = json.loads(temp_json)
    temp_key_list = list(temp_json.keys())
    temp_data = pd.json_normalize(temp_json[temp_key_list[0]]["row"])
    data_list.append(temp_data)
data = pd.DataFrame()
data = pd.concat(data_list)
